In [1]:
#| default_exp training

In [2]:
#|export
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
from pathlib import Path
from torch import tensor,nn
import torch.nn.functional as F
from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [3]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin#1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

In [4]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [8]:
n

50000

In [6]:
# Define a new class called `Model` that inherits from `nn.Module`.
class Model(nn.Module):
# Define a constructor method for the `Model` class that takes three arguments: `n_in`, `nh`, and `n_out`.
    def __init__(self, n_in, nh, n_out):
# Call the constructor of the parent class (`nn.Module`) to initialize the `Model` object.
        super().__init__()
# Create a list called `layers` that contains three elements:
  # A linear layer that maps `n_in` input features to `nh` output features.
  # A ReLU activation function.
  # A linear layer that maps `nh` input features to `n_out` output features.
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
# Define a method called `__call__` that takes an input tensor `x`.
    def __call__(self, x):
# Loop over each layer in the `layers` list and apply it to the input tensor `x`. 
# This effectively performs a forward pass through the neural network.
        for l in self.layers: 
            x = l(x)
# Return the output tensor `x` after passing it through all the layers in the neural network.
        return x

In [9]:
model = Model(m, nh, 10)
pred = model(x_train)
pred.shape

torch.Size([50000, 10])

$$\hbox{testing(x)}_{i,j}

First, we will need to compute the softmax of our activations. This is defined by:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{e^{x_{0}} + e^{x_{1}} + \cdots + e^{x_{n-1}}}$$

or more concisely:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum\limits_{0 \leq j \lt n} e^{x_{j}}}$$ 

In practice, we will need the log of the softmax when we calculate the loss.